# seedbleniding

exp40,exp49 のseed42,614


In [1]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

import pickle

In [6]:
train_42 = pd.read_csv("../output/exp40_lgb_lagdiff_c3/oof_exp40_lgb_0.7969750332783205_5fold_seed42.csv")
test_42 = pd.read_csv("../output/exp40_lgb_lagdiff_c3/test_exp40_lgb_0.7969750332783205_5fold_seed42.csv")

train_614 = pd.read_csv("../output/exp49_lgb_seed614/oof_exp49_lgb_0.7969425363638722_5fold_seed614.csv")
test_614 = pd.read_csv("../output/exp49_lgb_seed614/test_exp49_lgb_0.7969425363638722_5fold_seed614.csv")



In [7]:
train_42.head()

,customer_ID,target,exp40_lgb_oof
0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.839287
1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.013724
2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.998380
3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.002872
4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.001001


In [8]:
train_614.head()

,customer_ID,target,exp49_lgb_oof
0,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0.000761
1,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,0,0.219348
2,0000f99513770170a1aba690daeeb8a96da4a39f11fc27...,1,0.871894
3,0001337ded4e1c2539d1a78ff44a457bd4a95caa55ba17...,1,0.695717
4,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.790132


In [9]:
oof = train_42.merge(train_614,on = "customer_ID")
sub = test_42.merge(test_614,on = "customer_ID")


In [19]:
oof["exp42+49_oof"] = (oof["exp40_lgb_oof"] + oof["exp49_lgb_oof"])/2
sub["prediction"] = (sub["prediction_x"] + sub["prediction_y"])/2 

In [20]:
sub

,customer_ID,prediction_x,prediction_y,exp42+49_sub,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.026727,0.026443,0.026585,0.026585
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000746,0.000776,0.000761,0.000761
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.032303,0.033386,0.032845,0.032845
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.194742,0.205071,0.199907,0.199907
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.862604,0.869916,0.866260,0.866260
...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.013174,0.012070,0.012622,0.012622
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.831382,0.821951,0.826667,0.826667
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.426673,0.436741,0.431707,0.431707
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.265010,0.249526,0.257268,0.257268


In [18]:
oof

,customer_ID,target_x,exp40_lgb_oof,target_y,exp49_lgb_oof,exp42+49_oof
0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.839287,1,0.790132,0.814709
1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.013724,0,0.017299,0.015512
2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.998380,1,0.998302,0.998341
3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.002872,0,0.001653,0.002263
4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.001001,0,0.001061,0.001031
...,...,...,...,...,...,...
458908,fff908439f19e2b2a4eb26432d51d8e38a0b757515f0f3...,0,0.015025,0,0.013905,0.014465
458909,fffcb68b6284b022106c4a2726d7d1cee2a3b774f2e8da...,0,0.003548,0,0.004152,0.003850
458910,fffcfd4dcb5311e3b7022eeec6c194f0ee01528bb45a01...,0,0.002174,0,0.002211,0.002193
458911,fffdc0cf228085b4275b38ebe6eb915766af3fecb2ae28...,0,0.000177,0,0.000217,0.000197


In [22]:
oof[["customer_ID","exp42+49_oof"]].to_csv("exp40_oof_lgb_seedblend.csv",index = False)
sub[["customer_ID","prediction"]].to_csv("exp40_sub_lgb_seedblend.csv",index = False)


In [23]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


amex_metric(oof["target_x"], oof["exp42+49_oof"])

0.797531854076833